In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [5]:
transactions_df = pd.read_csv('creditcard_train.csv')

In [3]:
from imblearn.over_sampling import SMOTE

In [13]:
X_res, y_res = SMOTE().fit_sample(transactions_df.drop('Class', axis=1), transactions_df['Class'])
res_transactions_df = pd.concat((X_res, y_res), axis=1)

In [17]:
res_transactions_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,33419.000000,-2.178201,-3.132187,1.315758,-0.129783,-2.736013,0.743459,-0.752718,-2.650826,-0.184284,-1.392226,-1.114831,0.253591,-0.428280,-0.724290,-2.442338,0.649252,1.192440,-1.782696,0.014677,2.534626,-0.828762,-0.219136,-1.004913,0.788588,1.061994,-0.319407,-0.132313,0.333476,937.750000,0
1,151317.000000,2.064423,0.185575,-1.684612,0.411066,0.479555,-0.797963,0.205544,-0.240568,0.415454,-0.401418,-0.411296,0.439120,0.554022,-0.936324,0.196285,0.227611,0.510030,-0.465360,-0.031358,-0.130661,-0.351331,-0.876025,0.343288,0.522189,-0.259568,0.173623,-0.056280,-0.029665,1.980000,0
2,132434.000000,-0.547505,0.798072,-0.719939,-1.129561,0.925708,0.763338,0.231338,0.799204,-0.277812,-0.348452,0.511676,-0.034514,-1.390963,1.067757,0.423194,-0.262177,-0.003822,-0.601821,-0.988247,-0.313966,0.366664,1.068933,-0.101523,-1.604148,-0.318277,0.838076,0.012324,-0.015564,11.950000,0
3,81787.000000,-0.945710,0.323579,0.595681,-1.288095,0.818906,-0.748491,0.890076,-0.130671,-0.471365,-0.389743,0.537702,0.137711,-0.555964,0.457870,-0.277456,0.662419,-0.951232,-0.103400,0.466225,-0.263767,-0.371528,-1.149510,0.217859,-0.507989,-0.026857,0.591496,-0.326179,-0.007543,24.980000,0
4,125062.000000,1.898722,-0.321038,-1.771837,0.672408,0.115019,-1.267347,0.612810,-0.441070,0.450298,0.107004,-1.144233,-0.035721,-0.728277,0.565399,-0.368894,-0.494280,-0.107078,-0.658594,0.169095,-0.082106,0.015111,0.006269,-0.029094,-0.071333,0.179444,0.378225,-0.106042,-0.059506,104.360000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397995,11948.317802,-4.351654,3.467765,-6.383989,6.373145,-2.118322,-1.720818,-4.967646,1.077450,-1.971173,-5.396462,6.142152,-12.084335,0.402296,-9.484561,0.497078,-7.281358,-10.736946,-4.295617,1.228121,0.038137,0.794857,0.177889,0.441887,-0.372011,-0.328381,0.071096,-0.832982,0.763971,26.142719,1
397996,41563.607129,-11.004002,6.254466,-12.628058,7.162776,-9.575534,-1.707591,-11.731741,4.365579,-5.872360,-10.223044,4.539364,-9.046323,0.567731,-8.002257,0.678615,-8.992622,-15.337312,-7.066531,2.661543,-0.978365,1.777102,-0.562556,-1.393395,0.411258,-1.008284,-0.322712,-1.004433,-0.608051,284.438582,1
397997,81375.513504,-0.884634,1.791524,-0.949333,3.854829,-1.543857,0.187519,-2.990668,1.343978,-2.295150,-1.891322,1.907629,-4.006053,-0.936073,-2.811108,0.795319,-2.352171,-6.431643,-0.829999,1.731353,0.370555,0.746115,0.549486,-0.035275,-0.567933,-0.526273,0.258701,0.507699,0.177235,7.560990,1
397998,29484.797678,-0.303537,3.448520,-5.145009,4.790281,1.811835,-1.946380,0.667653,0.603985,-2.823833,-2.172252,3.811178,-1.668125,-1.192742,-6.994424,-0.290882,2.120081,5.405905,2.607732,-2.519857,0.051609,-0.016199,-0.338784,-0.559403,-0.419576,1.327724,0.481825,0.070503,0.164090,5.394058,1


In [23]:
import statsmodels.formula.api as smf

In [21]:
features = ['V' + str(i+1) for i in range(28) ]
formula = 'Class ~ ' + ' + '.join(features)

In [26]:
model = smf.ols(formula, res_transactions_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Class   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                 3.144e+04
Date:                Mon, 26 Oct 2020   Prob (F-statistic):               0.00
Time:                        11:41:42   Log-Likelihood:                -56671.
No. Observations:              398000   AIC:                         1.134e+05
Df Residuals:                  397971   BIC:                         1.137e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1571      0.001    273.744      0.000       0.156       0.158
V1            -0.0184      0.000    -77.404      0.000      -0.019      -0.018
V2             0.0042      0.000     13.930      0.000       0.004       0.005
V3            -0.0046      0.000    -16.025      0.000      -0.005      -0.004
V4             0.0542      0.000    158.685      0.000       0.054       0.055
V5             0.0024      0.000      7.298      0.000       0.002       0.003
V6            -0.0132      0.000    -32.169      0.000      -0.014      -0.012
V7             0.0292      0.000     89.909      0.000       0.029       0.030
V8            -0.0172      0.000    -91.059      0.000      -0.018      -0.017
V9             0.0084      0.000     18.867      0.000       0.008       0.009
V10           -0.0214      0.000    -54.776      0.000      -0.022      -0.021
V11            0.0019      0.000      3.945      0.000       0.001       0.003
V12            0.0178      0.000     44.601      0.000       0.017       0.019
V13           -0.0294      0.000    -62.015      0.000      -0.030      -0.029
V14           -0.0616      0.000   -187.032      0.000      -0.062      -0.061
V15            0.0013      0.001      2.438      0.015       0.000       0.002
V16            0.0065      0.001     11.376      0.000       0.005       0.008
V17           -0.0141      0.000    -38.124      0.000      -0.015      -0.013
V18            0.0357      0.001     57.415      0.000       0.034       0.037
V19            0.0151      0.001     27.500      0.000       0.014       0.016
V20            0.0274      0.001     45.252      0.000       0.026       0.029
V21            0.0170      0.000     48.630      0.000       0.016       0.018
V22            0.0292      0.001     41.974      0.000       0.028       0.031
V23           -0.0356      0.001    -68.858      0.000      -0.037      -0.035
V24            0.0190      0.001     21.607      0.000       0.017       0.021
V25           -0.0083      0.001     -9.676      0.000      -0.010      -0.007
V26           -0.0682      0.001    -64.558      0.000      -0.070      -0.066
V27            0.0038      0.001      6.106      0.000       0.003       0.005
V28            0.0510      0.001     41.585      0.000       0.049       0.053
==============================================================================
Omnibus:                    25952.695   Durbin-Watson:                   1.378
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31527.973
Skew:                           0.655   Prob(JB):                         0.00
Kurtosis:                       3.431   Cond. No.                         48.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is co

In [27]:
collinear_features = []
for feature in features:
    other_features = features.copy()
    other_features.remove(feature)
    feature_formula = feature + ' ~ ' + ' + '.join(other_features)
    feature_model = smf.ols(feature_formula, res_transactions_df).fit()
    VIF = 1/(1 - feature_model.rsquared)
    if VIF >= 10:
        collinear_features.append(feature)
    print(feature, VIF)

noncollinear_features = [feature for feature in features if feature not in collinear_features]
print(collinear_features)
print(noncollinear_features)

V1 8.138722973828493
V2 5.79956205882602
V3 15.289135538439266
V4 5.856811397096758
V5 8.740343185579516
V6 2.469816647478723
V7 17.273547245976435
V8 4.042552740362449
V9 5.05244675227656
V10 15.095596230906482
V11 8.33026087103937
V12 16.20875468562786
V13 1.167443373171677
V14 11.558536357306151
V15 1.2209344735858745
V16 18.23120327193029
V17 22.502748644822955
V18 10.307498243074432
V19 2.0447617594012844
V20 2.058687424469567
V21 4.438290763891195
V22 3.0650302212168556
V23 1.51084689147749
V24 1.1547244022923977
V25 1.4093518283461808
V26 1.1282578925689217
V27 1.7840253553021213
V28 1.3686928404871976
['V3', 'V7', 'V10', 'V12', 'V14', 'V16', 'V17', 'V18']
['V1', 'V2', 'V4', 'V5', 'V6', 'V8', 'V9', 'V11', 'V13', 'V15', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']


In [39]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [49]:
from sklearn.metrics import f1_score, recall_score, precision_score
def print_metrics(y_test, y_hat):
    print('recall', recall_score(y_test, y_hat))
    print('precision', precision_score(y_test, y_hat))
    print('F1', f1_score(y_test, y_hat))
X_train, y_train = res_transactions_df[noncollinear_features], res_transactions_df['Class']
test_df = pd.read_csv('creditcard_test.csv')
X_test, y_test = test_df[noncollinear_features], test_df['Class']


In [50]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [51]:
y_hat = rfc.predict(test_df[noncollinear_features])
print_metrics(y_test, y_hat)

recall 0.75
precision 0.7619047619047619
F1 0.7559055118110236


In [54]:
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)


AdaBoostClassifier()

In [56]:
y_hat = abc.predict(X_test)
print_metrics(y_test, y_hat)

recall 0.875
precision 0.03523120478137779
F1 0.06773510734804959
